In [2]:
import pandas as pd

In [7]:
data = pd.read_csv('/Users/camille/repo/Hetic/projet_gouv/scraping/Cleaning_eda/all_data_pos.csv')

In [8]:
data

,rate,review_text,object_address,cleaned_text,sentiment,pos_reviews_terms
0,4,Toujours très bien reçu surtout celle avec les...,"5 Rue de Quimper, 68100 Mulhouse",bien cheveux violet,1,[]
1,5,"Bonjour, En bref, le personnel fait son travai...","5 Rue de Quimper, 68100 Mulhouse",bonjour bref personnel travail question soluti...,1,['problèm']
2,4,"Tout le monde est top (agents comme public), l...","5 Rue de Quimper, 68100 Mulhouse",monde agent public ascenseur,1,"['agent', 'public', 'ascenseur']"
3,5,Nouveau bâtiment avec de la place sur le parki...,"5 Rue de Quimper, 68100 Mulhouse",bâtiment place parking min arret,1,"['place', 'parking']"
4,5,Ma Future 2eme maison,"5 Rue de Quimper, 68100 Mulhouse",future 2eme maison,1,[]
...,...,...,...,...,...,...
3649,5,Je tiens à remercier Mme Morand Sophie qui a é...,"45 Av. Billaud Varenne, 17000 La Rochelle",mme morand sophie interlocuteur 16/11/22 jour ...,1,"['interlocuteur', 'rendez-vous', 'prise', 'cha..."
3650,5,"Faut essaie de remonter les étoile un peux, Ok...","45 Av. Billaud Varenne, 17000 La Rochelle",étoile chose mieux partie conseiller bien écou...,1,"['chose', 'bon', 'humain']"
3651,5,En vu des commentaire je suis obligée de laiss...,"45 Av. Billaud Varenne, 17000 La Rochelle",commentaire avis couseilliere mme.bou écoute r...,1,"['rapide', 'changement', 'tomber', 'pole', 'em..."
3652,3,j'ai eu soixante ans le 15 mars ; suis je obli...,"45 Av. Billaud Varenne, 17000 La Rochelle",soixant mars pointé,1,['pointer']


In [9]:
data.dropna(subset=['review_text'], inplace=True)

In [12]:
data['pos'] = data['rate'].apply(lambda x:  x >= 2.5)
df_pos = data[data['pos'] == True].reset_index(drop=True)
df_neg = data[data['pos'] == False].reset_index(drop=True)

In [10]:
from bertopic import BERTopic



/Users/camille/repo/Hetic/projet_gouv/scraping/gouv_env/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/camille/repo/Hetic/projet_gouv/scraping/gouv_env/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/camille/repo/Hetic/pr

In [11]:
topic_model = BERTopic()
BERTopic(language="multilingual") 

## SentenceTransformer

In [13]:
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("paraphrase-albert-small-v2")
topic_model = BERTopic(embedding_model=sentence_model, nr_topics=5)
sentence_model = SentenceTransformer("paraphrase-albert-small-v2")
topic_model_pos = BERTopic(embedding_model=sentence_model, nr_topics=5)
data['cleaned_text'] = data['cleaned_text'].astype(str)
data.dropna(subset=['cleaned_text'], inplace=True)
data.reset_index(drop=True, inplace=True)
topics, probs = topic_model_pos.fit_transform(data['cleaned_text'])
topic_model_pos.get_topic_info()

In [14]:
sentence_model = SentenceTransformer("paraphrase-albert-small-v2")
topic_model_pos = BERTopic(embedding_model=sentence_model, nr_topics=5)
data['cleaned_text'] = data['cleaned_text'].astype(str)
data.dropna(subset=['cleaned_text'], inplace=True)
data.reset_index(drop=True, inplace=True)
topics, probs = topic_model_pos.fit_transform(data['cleaned_text'])
topic_model_pos.get_topic_info()

,Topic,Count,Name
0,-1,1471,-1_conseiller_emploi_accueil_bon
1,0,2024,0_bon_bien_accueil_agence
2,1,89,1_nan___
3,2,58,2_réactif_écoute_personnel_conseiller
4,3,12,3_disponible_écoute_personnel_extrêmement


In [16]:
topic_model_pos.visualize_barchart()

In [15]:
topics_to_merge = [-1, 1]
topic_model_pos.merge_topics(data['cleaned_text'], topics_to_merge)


In [21]:
#Save in csv 
topic_model_pos = topic_model_pos.get_document_info(data['cleaned_text']).to_csv('/Users/camille/repo/Hetic/projet_gouv/scraping/topic/bert_pos.csv', index=False, sep=';')

In [55]:
sentence_model = SentenceTransformer("paraphrase-albert-small-v2")
topic_model_neg = BERTopic(embedding_model=sentence_model, nr_topics=5)
df_neg['cleaned_text'] = df_neg['cleaned_text'].astype(str)
df_neg.dropna(subset=['cleaned_text'], inplace=True)
df_neg.reset_index(drop=True, inplace=True)
topics, probs = topic_model_neg.fit_transform(df_neg['cleaned_text'])
topic_model_neg.get_topic_info()

,Topic,Count,Name
0,-1,3065,-1_emploi_conseiller_pôle_rien
1,0,2626,0_conseiller_emploi_rien_pôle
2,1,87,1_nan___
3,2,68,2_temps_numéro_commentaire_temp
4,3,38,3_parking_place_petit_retard


In [64]:
topics_to_merge = [-1, 1]
topic_model_neg.merge_topics(df_neg['cleaned_text'], topics_to_merge)


In [53]:
topic_model_neg.get_topic_info()

,Topic,Count,Name
0,-1,2978,-1_emploi_conseiller_pôle_rien
1,0,2778,0_conseiller_emploi_rien_pôle
2,1,89,1_nan_mappa_esclavagisme_
3,2,39,2_parking_place_petit_retard


In [62]:
topic_model_neg.get_topic(3)


[('parking', 0.4849276449158877),
 ('place', 0.1646501623920943),
 ('petit', 0.0850114350770381),
 ('retard', 0.07428304584713541),
 ('accès', 0.0679348350177674),
 ('local', 0.060597821575057984),
 ('minute', 0.060414870171001674),
 ('trottoir', 0.060098737802666245),
 ('bâtiment', 0.05919626284528834),
 ('grand', 0.05218097349245045)]

In [65]:
topic_model_neg.visualize_barchart()

In [68]:
topic_model_neg = topic_model_neg.get_document_info(df_neg['cleaned_text']).to_csv('topic/topic_model_neg.csv')